In [1]:
import sys
sys.path.append('../')

In [2]:
import cv2 as cv
import torch
import numpy as np
from sort.sort import Sort
import time
import matplotlib.pyplot as plt

In [3]:
from streetstudy.data import virat
from streetstudy.common import display
from streetstudy.common import evaluation
from streetstudy.common import postprocess
from streetstudy.common import utils
from streetstudy.model import yolo

In [4]:
video_path = "/mnt/d/data/virat/videos/VIRAT_S_000002.mp4"

tick = time.time()

video_dict = utils.get_video_metadata(video_path)
model = yolo.YoloModel(conf=0.2)
frame_iterator = iter(utils.frame_generator(path=video_dict['path']))

for frame_number in range(video_dict['length']):
    frame = next(frame_iterator)
    model.predict(frame, frame_number)
        
tock = time.time()
print(f"{tock - tick:.4f} seconds")

preds = model.predictions
print(preds)

%matplotlib inline

def plot_analysis(video_width, video_height, preds, frame, frame_number):
    fig, axs = plt.subplots(2,2, layout="constrained", figsize=(10,5))

    for row in range(2):
        for col in range(2):
            axs[row, col].set(xlim=(0, video_width), ylim=(video_height, 0))    
            axs[row, col].set_axis_off()
            axs[row, col].set_facecolor("b")

    postprocess.heatmap(axs[0,0], preds, frame_number)
    postprocess.bounding_boxes(axs[0,1], preds, frame_number)
    postprocess.directional_arrows(axs[1,0], preds, frame_number)

    axs[0,0].imshow(cv.cvtColor(frame, cv.COLOR_BGR2RGB))
    axs[0,1].imshow(cv.cvtColor(frame, cv.COLOR_BGR2RGB))
    axs[1,0].imshow(cv.cvtColor(frame, cv.COLOR_BGR2RGB))
    
    fig.show()

plot_analysis(video_dict['width'], video_dict['height'], preds, frame, frame_number)

# Number of unique people in video
unique_objs = np.unique((preds[:, 5]))
num_unique = len(unique_objs)
print(num_unique)

## Dwell Times Through Binary Classification

In [5]:
video_path

'/mnt/d/data/virat/videos/VIRAT_S_000002.mp4'

In [10]:
dwelling_events = [1,2,3,4,5,6,7,8]

virat_df = virat.get_dataset_df()
virat_anno_df = virat.get_annotations_df(video_path, type='events', object_id=True)
virat_anno_df = virat_anno_df[['current_frame', 'bbox_lefttop_x', 'bbox_lefttop_y', 'bbox_rightbottom_x', 'bbox_rightbottom_y', 'event_type']]
virat_anno_df['dwelling'] = 0
print(virat_anno_df.columns)

Index(['current_frame', 'bbox_lefttop_x', 'bbox_lefttop_y',
       'bbox_rightbottom_x', 'bbox_rightbottom_y', 'event_type', 'dwelling'],
      dtype='object')


In [11]:
virat_anno_df['event_type'].unique()

array([6, 3, 4, 5, 2, 9])

In [12]:
for _, row in virat_anno_df.iterrows():
    if row['event_type'] in dwelling_events:
        row['dwelling'] = 1
    else:
        row['dwelling'] = 0